# How is this competition scored?
To do well in this competition it is very important to understand the algorithm by which the competition is scored - The Kullback Leibler Divergence.

Here I have borrowed the code from Kaggle's example scoring script (https://www.kaggle.com/code/metric/kullback-leibler-divergence/notebook), and used it to create a notebook where you can play around with the probability scoring to improve your understanding.

There's a useful discussion posted here: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/466731 by MARÍLIA PRATA.

In [ ]:
import pandas as pd
import numpy as np
import warnings

In [ ]:
import metric

^ For this import code to run make sure you've gone to the far right of the notebook and have kullback-leibler-divergence in your Input. Otherwise use the "+ Add Data" button to add it. You may also need kaggle_metric_utilities.

I struggled to get this to work in my other notebook, probably because the Internet setting is required to be turned off for entry submission.

In [ ]:
# Filter out SettingWithCopyWarning
warnings.filterwarnings("ignore")
# Setup useful functions
vote_list = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
def valid_submission(submission_df):
    for i, eeg_id in enumerate(submission_df['eeg_id']):
        vote_total = 0
        for vote in vote_list:
            vote_total += submission_df[vote].iloc[i]
        if 0.99 > vote_total > 1.01:
            return False
    return True

Let's define 3 'types' of probability distributions:
* 1 certainty
* 2 main posibilities
* 6 evenly weighted probabilities

Then we will compare each combination of these for both the submission and solution.

In [ ]:
submission_dict = {
    'eeg_id': [0, 1, 2, 3, 4, 5, 6, 7, 8],
    'seizure_vote': [1, 1, 1, 1/2, 1/2, 1/2, 1/6, 1/6, 1/6],
    'lpd_vote': [0, 0, 0, 1/2, 1/2, 1/2, 1/6, 1/6, 1/6],
    'gpd_vote': [0, 0, 0, 0, 0, 0, 1/6, 1/6, 1/6],
    'lrda_vote': [0, 0, 0, 0, 0, 0, 1/6, 1/6, 1/6],
    'grda_vote': [0, 0, 0, 0, 0, 0, 1/6, 1/6, 1/6],
    'other_vote': [0, 0, 0, 0, 0, 0, 1/6, 1/6, 1/6],
}
solution_dict = {
    'eeg_id': [0, 1, 2, 3, 4, 5, 6, 7, 8],
    'seizure_vote': [1, 1/2, 1/6, 1, 1/2, 1/6, 1, 1/2, 1/6],
    'lpd_vote': [0, 1/2, 1/6, 0, 1/2, 1/6, 0, 1/2, 1/6],
    'gpd_vote': [0, 0, 1/6, 0, 0, 1/6, 0, 0, 1/6],
    'lrda_vote': [0, 0, 1/6, 0, 0, 1/6, 0, 0, 1/6],
    'grda_vote': [0, 0, 1/6, 0, 0, 1/6, 0, 0, 1/6],
    'other_vote': [0, 0, 1/6, 0, 0, 1/6, 0, 0, 1/6],
}
submission_df = pd.DataFrame(submission_dict)
solution_df = pd.DataFrame(solution_dict)
print(f"Submission df is valid: {valid_submission(submission_df)}")
print(f"Solution df is valid: {valid_submission(solution_df)}")

# Check the values are ok. Fill remaining probability with other.
print("\nIndividual row scores:")
for i, eeg_id in enumerate(solution_dict['eeg_id']):
    print(f"Score {i+1}: {metric.score(solution_df[solution_df['eeg_id'] == eeg_id], submission_df[submission_df['eeg_id'] == eeg_id], 'eeg_id')}")

print(f"\nOverall score: {metric.score(solution_df, submission_df, 'eeg_id')}")

It seems to me that it's nearly always best to play it safe with an even spread of probabilities in your submission (Scores 7-9), since they're guaranteed to be below 1.8.

Setting a high probability is very risky if you're wrong!